In [1]:
conda install -c rdkit rdkit scikit-learn -y

Channels:
 - rdkit
 - defaults
 - conda-forge
Platform: win-64
Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [1]:
from rdkit import Chem
from rdkit.Chem import Descriptors
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('train.csv')
mols = [Chem.MolFromSmiles(mol) for mol in df['SMILES_canonical']]
X = [[d[1](m) for d in Descriptors._descList] for m in mols]
Y = df['target_feature']



In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from pprint import pprint

#splitting data in training and testing data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

#Random forrest algorithm
forest = RandomForestClassifier(max_depth=3, random_state=0)
forest.fit(X_train, Y_train)

#Define feature importances
importances = forest.feature_importances_

#Create dictionary based on feature importance
high_coef = {}
for i, importance in enumerate(importances):
    feature_name = Descriptors._descList[i][0]
    high_coef[feature_name] = float(importance)

#Sort dictionary from highest importance to lowest
sorted_high_coef = dict(sorted(high_coef.items(), key=lambda item: item[1], reverse=True))

pprint(sorted_high_coef, sort_dicts=False)

{'MaxEStateIndex': 0.2871550597621642,
 'MaxAbsEStateIndex': 0.06428823521725728,
 'NumSaturatedRings': 0.014259293992323093,
 'Kappa1': 0.011929579855495254,
 'BCUT2D_MWLOW': 0.011823174853044165,
 'EState_VSA5': 0.011295975445671548,
 'Kappa3': 0.010812885962199852,
 'PEOE_VSA11': 0.010003329334773032,
 'FpDensityMorgan1': 0.009824123304410877,
 'TPSA': 0.009111944860674476,
 'fr_halogen': 0.009089537178803427,
 'PEOE_VSA8': 0.008951289503787069,
 'VSA_EState10': 0.008950116980688707,
 'fr_Al_OH_noTert': 0.008864120819590092,
 'FpDensityMorgan3': 0.008526569512428434,
 'fr_pyridine': 0.00808149775765886,
 'fr_imide': 0.008023566090728297,
 'fr_phos_acid': 0.007856873718958815,
 'fr_unbrch_alkane': 0.0077171246631059206,
 'fr_Ar_OH': 0.007247277652637259,
 'fr_aniline': 0.007133610211016895,
 'fr_NH1': 0.007045469095292574,
 'PEOE_VSA4': 0.0070388751825492145,
 'MaxPartialCharge': 0.006859315045897823,
 'Chi3n': 0.006789551799048426,
 'NumHAcceptors': 0.006577806969062281,
 'MinAbsPar